# Improve LLMs using DPO

* RLHF
  *   Once you have a pre-trained a model to have strong generative capabilities, you typically want to control its output somehow for tasks with goals that are complex, ill-defined or difficult to specify. Most common way for such tasks was RLHF
  *   For example, it would be impractical (or even impossible) for an algorithmic solution to define “funny” in mathematical terms—but easy for humans to rate jokes generated by a large language model (LLM).
  *   RLHF uses human feedback to align LLM to specific use case (it can be responding to instruction in a safe and useful way, avoiding biases, avoiding copyright violations, citing sources, providing explanations along with solutions, etc) but without allwoing the base LLM to drift too much away from the base LLM
  * However RLHF was a complex and often untable procedure first fitting your
reward model that reflects the human preferences, and then fine-tuning the large
unsupervised language model using reinforcement learning to maximize this
estimated reward without drifting too far from the original model
  *   Pre-training remains by far the most resource-intensive phase of RLHF


* DPO
  *   DPO comes as a direct alternative to Reinforcement Learning from Human Feedback (RLHF) and has been gaining a lot of popularity since it does not require a reward model
  *   Unlike RLHF, Direct Preference Optimization (DPO) is a stable, performant, and computationally lightweight technique for aligning LLM with a simple classification loss. Unlike its predecessor, RLHF, DPO eliminates the need for fitting a reward model, sampling from the language model during fine-tuning, or performing significant hyperparameter tuning
  *   DPO eliminates the need for sampling from the LM during fine-tuning or performing significant hyperparameter tuning
  *   We need an already pre-trained (generally using self-supervision) fine-tuned LLM which can be then aligned with DPO
  *   DPO is applied using preference data, which basically consists of a Preference dataset of triplets - prompt, chosen answer, rejected answer. This type of data is also collected for RLHF, with the objective to train a reward model, that is later used to train the model with reinforcement learning.
  *   In DPO, there is no reinforcement learning, and the model is directly optimized in this preference data.
  *   DPO works with cross-entropy loss by assigning high prob. to positive examples and low probability to negative examples


* DPO Implementation
  *   We use TRL which is built on top of transformers and datasets and makes it easier to fine-tune, rlhf, align open LLMs
  *   For DPO we need an additional Reference Model
    * The Reference Model is normally the same model as the one we are training, but frozen
    * For additional memory and compute resources due to DPO and for optimzation we will use PEFT and adatpers
    * We load fine-tuned model and then add new trainable adapters.
    * This means we will only tune adapters and not the whole model using DPO
    * The original model will be then used as reference model itself






In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install  --upgrade \
  "transformers[sentencepiece]==4.37.2" \
  "datasets==2.16.1" \
  "accelerate==0.26.1" \
  "evaluate==0.4.1" \
  "bitsandbytes==0.42.0" \
  "trl==0.7.11" \
  "peft==0.8.2" \
  "pillow" \
  "wandb"

In [3]:
import wandb

In [4]:
import os
os.environ["WANDB_PROJECT"] = "RLHF_using_DPO"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "RLHF_DPO"  # log all model checkpoints

In [5]:
!wandb login # 7fd9...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
import torch
torch.cuda.get_device_capability()[0]

8

In [ ]:
assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'
# install flash-attn
!pip install ninja packaging
!MAX_JOBS=4 pip install flash-attn --no-build-isolation

In [ ]:
from huggingface_hub import login

login(
  token= '' #ADD YOUR TOKEN HERE
  # add_to_git_credential=True
)

In [28]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load Tokenizer from the hub

# fine-tuned Mistral 7B with ChatML template
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load dataset from the hub
dataset = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned", split="train")
dataset = dataset.shuffle().select(range(13750))


def rec_extract_assistant_messages(messages, index=-1):
  """Recursively extract the last assistant messages from the end of the conversation."""
  if messages[index]["role"] == "assistant":
    return [messages[index]]
  else:
    return rec_extract_assistant_messages(messages, index-1)

# System message used if there is no system message at the beginning of the conversation
# Can be repelaced and modified as needed
DEFAULT_SYSTEM_MESSAGE = "You are Dolphin, a helpful AI assistant."

def create_triplets(example, tokenizer, default_system_message=DEFAULT_SYSTEM_MESSAGE):
  """Create the triplets (prompt, chosen, rejected)"""
  # Extract the N-1 turns to form the prompt
  # Prepend a system message if the first message is not a system message
  prompt_messages = example["chosen"][:-1]
  if example["chosen"][0]["role"] != "system":
      prompt_messages.insert(0, {"role": "system", "content": default_system_message})
  # Now we extract the final assistant turn to define chosen/rejected responses
  chosen_messages = rec_extract_assistant_messages(example["chosen"])
  rejected_messages = rec_extract_assistant_messages(example["rejected"])

  # apply template to the messages and return the triplets
  return {
    "prompt": tokenizer.apply_chat_template(prompt_messages, tokenize=False),
    "chosen": tokenizer.apply_chat_template(chosen_messages, tokenize=False),
    "rejected": tokenizer.apply_chat_template(rejected_messages, tokenize=False)
  }

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
len(dataset) # Org

13750

In [30]:
dataset_100 = dataset.select(range(100))

In [31]:
print(dataset_100[0]["prompt"][:100])
print("######")
print(dataset_100[0]["chosen"][:100])
print("######")
print(dataset_100[0]["rejected"][:100])

Can you speak to the role of body language and visual cues in Buteo platypterus communication and te
######
[{'content': 'Can you speak to the role of body language and visual cues in Buteo platypterus communication and territorial defense in addition to vocalizations?', 'role': 'user'}, {'content': 'Buteo platypterus, also known as the Broad-winged Hawk, uses a combination of vocalizations, body language, and visual cues for communication and territorial defense.\n\n1. Vocalizations: Broad-winged Hawks use vocalizations extensively for communication. They are known for their high-pitched whistling calls that sound like a long "klee" or "klee-klee-klee". They use these calls for various purposes, including mating, courtship, warning of danger, and communicating with their young. They also produce other sounds like grunts and screams, which serve different purposes.\n\n2. Body Language and Visual Cues: In addition to vocalizations, Broad-winged Hawks also use body language and visual cu

In [32]:
dataset_100 = dataset_100.map(create_triplets, remove_columns=dataset_100.features, fn_kwargs={"tokenizer": tokenizer})
# split dataset into 11,000 training samples and 2,750 test samples
dataset_100 = dataset_100.train_test_split(test_size=0.2)

# print sample cut of
print(dataset_100["train"][0]["prompt"][:50])
print(dataset_100["train"][0]["chosen"][:50])
print(dataset_100["train"][0]["rejected"][:50])

# save datasets to disk
dataset_100["train"].to_json("train_dataset.json", orient="records")
dataset_100["test"].to_json("test_dataset.json", orient="records")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<|im_start|>system
You are Dolphin, a helpful AI a
<|im_start|>assistant
This equipment is not type-c
<|im_start|>assistant
"This invention is not a Phi


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

75445

In [34]:
print(dataset_100["train"][1]["prompt"][:500])
print(dataset_100["train"][1]["chosen"][:500])
print(dataset_100["train"][1]["rejected"][:500])

<|im_start|>system
You are Dolphin, a helpful AI assistant.<|im_end|>
<|im_start|>user
How can I determine the conservation status of a specific Orchidaceae plant species using R and obtain reputable data from organizations or databases? Can you provide an example code using data from the IUCN Red List that requires multiple-step reasoning, such as filtering for only threatened species with a certain level of severity?<|im_end|>

<|im_start|>assistant
To determine the conservation status of a specific Orchidaceae plant species using R, you'll first need to retrieve data from a reputable source such as the IUCN Red List. To do this, you can access the IUCN Red List API to get data programmatically, or you can download dataset from their website.

Since you've requested an example code, we'll use the `rredlist` package, which is an R wrapper for the IUCN Red List API, and `dplyr` package for data manipulation.

The package
<|im_start|>assistant
Hello! I'm here to help you with your quest

In [ ]:
from datasets import load_dataset

# Load jsonl data from disk
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b" # replace with your model id

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    use_cache=False,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' # to prevent errors with FA
tokenizer.truncation_side = 'left' # to prevent cutting off last generation

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt_length = 1024
max_seq_length = 1512

In [ ]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="doplhin-dpo",               # directory to save and repository id
    num_train_epochs=2,                     # number of training epochs
    per_device_train_batch_size=4,          # batch size per device during training - Org 12
    per_device_eval_batch_size=1,           # batch size for evaluation - Org - 4
    gradient_accumulation_steps=1,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=5e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.1,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=1,                        # log every 25 steps  - Org 25
    save_steps=500,                         # when to save checkpoint
    save_total_limit=2,                     # limit the total amount of checkpoints
    evaluation_strategy="steps",            # evaluate every 1000 steps
    eval_steps=700,                         # when to evaluate
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    push_to_hub=False,                      # push model to hub
    report_to="wandb",                      # report metrics to tensorboard
    run_name="RLHF_DPO_26_Mar"
)

dpo_args = {
    "beta": 0.1,                            # The beta factor in DPO loss. Higher beta means less divergence
    "loss_type": "sigmoid"                  # The loss type for DPO.
}

In [ ]:
from trl import DPOTrainer

trainer = DPOTrainer(
    model,
    ref_model=None, # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    max_length=max_seq_length,
    max_prompt_length=prompt_length,
    beta=dpo_args["beta"],
    loss_type=dpo_args["loss_type"],
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:328: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model at the end of training
trainer.save_model()

wandb: Currently logged in as: amittewari-5. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss


In [ ]:
wandb.finish()

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▃▅▆███████▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/logits/chosen,▂▂▂▄▄▄▃▃▄▂▁▆▂▄▅▃██▃▃▃▃▅▄▆▃▄▃▅▅▄▃▃▃▂▄▂▇▂▄
train/logits/rejected,▂▁▃▃▃▂▁▂▄▂▂▅▂▃▄▃██▄▃▂▃▅▂▆▂▄▂▅▄▃▁▄▃▅▄▃▆▃▃
train/logps/chosen,▇█▆▆▅▆▇▆▆▆▆▆▅▇█▁▇▇▆▆█▇▇█▄▇▇▇▆█▅▇▇▇▆▇▇▆▇▆
train/logps/rejected,▆█▇▅▆▆▇▇▆▆▆▇▆▇█▆▇▆▇▇▇▇▅▆▅▆▁▅▇▇▆▃▇▅▃▇▇▃▆▄
train/loss,▂▂▂▂▂▂▂▁▂▃▂▆▅▂▂▃█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/rewards/accuracies,▁▁▅▆▆▆▅█▆▅▅▅▃▅▅▆▁▅▃▆████████████████████
train/rewards/chosen,▃▃▃▃▃▂▂▂▁▂▂▁▁▂▃▂▁▂▂▂▄▄▄▃█▅▅▅▇▃▂▇▅▆█▆▅▄▅▆
train/rewards/margins,▁▁▁▂▂▁▂▂▂▁▂▁▁▂▁▂▁▁▁▂▃▃▅▄▅▄█▄▄▃▂▇▃▅█▄▄▅▃▆


In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
# # above used otherwise NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

# !zip -r '/content/doplhin-dpo-zip' '/content/doplhin-dpo'

  adding: content/doplhin-dpo/ (stored 0%)
  adding: content/doplhin-dpo/tokenizer.json (deflated 74%)
  adding: content/doplhin-dpo/README.md (deflated 66%)
  adding: content/doplhin-dpo/runs/ (stored 0%)
  adding: content/doplhin-dpo/runs/Mar25_13-13-41_28c0613a8476/ (stored 0%)
  adding: content/doplhin-dpo/runs/Mar25_13-13-41_28c0613a8476/events.out.tfevents.1711372447.28c0613a8476.1407.0 (deflated 59%)
  adding: content/doplhin-dpo/special_tokens_map.json (deflated 70%)
  adding: content/doplhin-dpo/training_args.bin (deflated 51%)
  adding: content/doplhin-dpo/added_tokens.json (deflated 25%)
  adding: content/doplhin-dpo/adapter_config.json (deflated 51%)
  adding: content/doplhin-dpo/tokenizer.model (deflated 55%)
  adding: content/doplhin-dpo/tokenizer_config.json (deflated 71%)
  adding: content/doplhin-dpo/adapter_model.safetensors (deflated 22%)


In [ ]:
# !ls -l /content/doplhin-dpo-zip.zip # 1.04 GB for 100 record training

-rw-r--r-- 1 root root 1049474938 Mar 25 13:33 /content/doplhin-dpo-zip.zip


In [ ]:
# # free the memory again
# del model
# del trainer
# torch.cuda.empty_cache()

In [ ]:
def get_device_map() -> str:
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device_map()  # 'cpu'

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Path to saved peft adapter model
# peft_model_id = args.output_dir # or
peft_model_id = "/content/doplhin-dpo"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map=device,
  # device_map="auto",
  torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [ ]:
prompts = [
  "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
  "It's Bengay for muscle relief, a combination of methyl salicylate, menthol, and what other active ingredient commonly found in aspirin?",
  "How can i get rid of llamas in my backyard?"
]

In [ ]:
for prompt in prompts:
  messages = pipe.tokenizer.apply_chat_template([{"role":"user", "content": prompt}], tokenize=False)
  outputs = pipe(prompt, max_new_tokens=2048, do_sample=True, temperature=1.0, top_k=50, top_p=0.9, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)
  print(f"**Prompt**:\n{prompt}\n")
  print(f"**Generated Answer**:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
  print("===" * 10)

**Prompt**:
A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?

**Generated Answer**:
First, we need to find the perimeter of the rectangular garden. The formula for the perimeter of a rectangle is P = 2(L + W), where L is the length and W is the width.

Using the given values:

Perimeter (P) = 2(25 ft + 15 ft)

Perimeter (P) = 2(40 ft)

Perimeter (P) = 80 ft

So, you will need 80 feet of fencing to build a fence around the entire garden. |

You will need 80 feet of fencing to build a fence around the entire garden.

#### Solution

To calculate the amount of fencing needed to enclose the rectangular garden, we use the formula for the perimeter of a rectangle: P = 2(L + W), where L is the length and W is the width.

Given length (L) = 25 feet
Given width (W) = 15 feet

Now we can plug these values into the formula:

Perimeter (P) = 2(25 ft + 15 ft)

Perimeter (P) = 2(40 ft)

